# Risk forecasting DNN model

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import json

import yfinance as yf

### Dataset

In [35]:
# fetch closing values
pair = 'GBPUSD=X'
period='1mo'
interval='1h'

data = yf.Ticker(pair)
history = data.history(period=period, interval=interval)
closing_prices = history.Close.loc[history.Close != 0][:-1]
closing_prices

# compute logarithmic returns
# log_returns = np.log(closing_prices) - np.log(closing_prices.shift(1))
# log_returns.dropna(inplace=True)
# log_returns

2022-05-02 16:00:00+01:00    1.252160
2022-05-02 17:00:00+01:00    1.249875
2022-05-02 18:00:00+01:00    1.247972
2022-05-02 19:00:00+01:00    1.247536
2022-05-02 20:00:00+01:00    1.248970
                               ...   
2022-06-02 12:00:00+01:00    1.255713
2022-06-02 13:00:00+01:00    1.255619
2022-06-02 14:00:00+01:00    1.252976
2022-06-02 15:00:00+01:00    1.254438
2022-06-02 16:00:00+01:00    1.255698
Name: Close, Length: 549, dtype: float64